## Approach 1 (Using Only HF)

### Setup

In [32]:
# Imports
from transformers import pipeline
import torch

from PIL import Image
from IPython.display import Audio as IPythonAudio

### 1. Image to object detection

In [33]:
# Specify the inference task
classifier = pipeline(
    task="object-detection", 
    model="facebook/detr-resnet-50",
)

Loading weights:   0%|          | 0/530 [00:00<?, ?it/s]

DetrForObjectDetection LOAD REPORT from: facebook/detr-resnet-50
Key                                                            | Status     |  | 
---------------------------------------------------------------+------------+--+-
model.backbone.model.layer3.0.downsample.1.num_batches_tracked | UNEXPECTED |  | 
model.backbone.model.layer4.0.downsample.1.num_batches_tracked | UNEXPECTED |  | 
model.backbone.model.layer2.0.downsample.1.num_batches_tracked | UNEXPECTED |  | 
model.backbone.model.layer1.0.downsample.1.num_batches_tracked | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


### 2. Extract labels 

In [36]:
traffic_img = Image.open("images/traffic.jpeg")

detections = classifier(traffic_img)
detections

[{'score': 0.682793378829956,
  'label': 'car',
  'box': {'xmin': 1904, 'ymin': 922, 'xmax': 2016, 'ymax': 1025}},
 {'score': 0.9888835549354553,
  'label': 'car',
  'box': {'xmin': 346, 'ymin': 1726, 'xmax': 1081, 'ymax': 2286}},
 {'score': 0.5736232399940491,
  'label': 'person',
  'box': {'xmin': 1843, 'ymin': 831, 'xmax': 1948, 'ymax': 942}},
 {'score': 0.8973760008811951,
  'label': 'car',
  'box': {'xmin': 1807, 'ymin': 1124, 'xmax': 1937, 'ymax': 1291}},
 {'score': 0.5430463552474976,
  'label': 'person',
  'box': {'xmin': 1464, 'ymin': 1192, 'xmax': 1555, 'ymax': 1277}},
 {'score': 0.9305405616760254,
  'label': 'bicycle',
  'box': {'xmin': 2264, 'ymin': 1505, 'xmax': 2368, 'ymax': 1646}},
 {'score': 0.5767219066619873,
  'label': 'person',
  'box': {'xmin': 270, 'ymin': 1647, 'xmax': 391, 'ymax': 1796}},
 {'score': 0.6837691068649292,
  'label': 'truck',
  'box': {'xmin': 250, 'ymin': 1184, 'xmax': 1065, 'ymax': 1723}},
 {'score': 0.5526701807975769,
  'label': 'person',
  'bo

### 3. Generate text

In [21]:
def summarize_detections(detections):
    counts = {}

    for d in detections:
        counts[d["label"]] = counts.get(d["label"], 0) + 1

    return "The image contains " + ", ".join(
        f"{v} {k}{'s' if v > 1 else ''}" for k, v in counts.items()
    ) + "."

In [49]:
text = summarize_detections(detections)

print(text)

The image contains 29 cars, 21 persons, 2 bicycles, 5 trucks, 1 bus.


### 4. Text to speech

In [26]:
# Specify the inference task
narrator = pipeline(
    task="text-to-speech",
    model="suno/bark-small",
)

config.json: 0.00B [00:00, ?B/s]

D:\Users\msonti\Source\AI LABS\HuggingFace_Exp\.env_hf\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\msonti\.cache\huggingface\hub\models--suno--bark-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/542 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.1.weight to fine_acoustics.lm_heads.0.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.2.weight to fine_acoustics.lm_heads.1.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.3.weight to fine_acoustics.lm_heads.2.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_l

...

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

In [38]:
narrated_text = narrator(text)
print(narrated_text["sampling_rate"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=60) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=60) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

24000


### 5. Audio output

In [31]:
IPythonAudio(
    narrated_text["audio"],
    rate=narrated_text["sampling_rate"]
)

## Approach 2 (Using Gemini and HF)

In [40]:
! pip install google-genai

  Using cached google_auth-2.48.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached tenacity-9.1.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached websockets-15.0.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp312-cp312-win_amd64.whl.metadata (7.4 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached pyasn1-0.6.2-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/724.7 kB ? eta -:--:--
   ---------------------------- ----------- 524.3/724.7 kB 4.2 MB/s eta 0:00:01
   ------------------------


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. Image Captioning using Google GenAI SDK

In [46]:
from google import genai
from google.genai import types

f = open("keys/.gemini.txt")
key = f.read()

client = genai.Client(api_key=key)

In [65]:
with open('Images/traffic.jpeg', 'rb') as f:
  image_bytes = f.read()

context = [
    types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/jpeg',
    ),
    types.Part.from_text(
        text="You are a helpful AI Assistant. Given an image, perform object detection and provide a text output that contains the information about the labels detected and their counts."
    )
]

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=context,
)

response_text = response.text
print(response_text)

I can see 56 objects in this image: 51 cars, 2 persons, 1 truck, 1 motorcycle, and 1 bus.


### 2. Text preprocessing


In [50]:
import re

def clean_text(text):
    text = text.strip()  # remove leading/trailing spaces
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s,.:]', '', text)  # remove unwanted symbols
    return text

In [51]:
def improve_grammar(text):
    text = text.replace("According to the image,", "").strip()
    
    # Convert "are present" → "there are"
    if "are present" in text:
        text = text.replace("are present", "")
        text = "There are " + text.strip()
    
    return text

In [54]:
! pip install num2words

from num2words import num2words
import re

def numbers_to_words(text):
    def replace_number(match):
        return num2words(int(match.group()))
    
    return re.sub(r'\d+', replace_number, text)

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13858 sha256=e4a46f2ed81e75b6bd3fb60f154db0b640d8c52db90af6ae97148664a0f1765b
  Stored in directory: c:\users\msonti\appdata\local\pip\cache\wheels\1a\bf\a1\4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
def adjust_length(text, mode="brief"):
    if mode == "brief":
        return text
    
    if mode == "detailed":
        return text + ". This suggests a busy traffic scene."

In [56]:
def add_intro(text, style="neutral"):
    if style == "neutral":
        return "Here is what I see in the image: " + text
    
    if style == "friendly":
        return "Here’s what I can see: " + text
    
    return text

In [62]:
def preprocess_caption(text, mode="detailed", tone="neutral"):
    text = clean_text(text)
    text = improve_grammar(text)
    text = numbers_to_words(text)
    text = adjust_length(text, mode)
    text = add_intro(text, tone)
    return text

In [66]:
final_text = preprocess_caption(response_text)
final_text

'Here is what I see in the image: I can see fifty-six objects in this image: fifty-one cars, two persons, one truck, one motorcycle, and one bus.. This suggests a busy traffic scene.'

### 3. Text to speech

In [68]:
narrated_final_text = narrator(final_text)
print(narrated_final_text["sampling_rate"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=60) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=60) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

24000


### 4. Audio output

In [69]:
IPythonAudio(
    narrated_final_text["audio"],
    rate=narrated_final_text["sampling_rate"]
)